In [1]:
#!pip install Census
#!pip install us

# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import us
from census import Census
from us import states
#from census_data import 

# Census API Key
from config import api_key
api_key
c = Census(api_key, year=2013)
c1 = Census(api_key,year=2017)

In [35]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", 
                          "B19013_001E", 
                          "B01003_001E", 
                          "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,08518,5217.0,41.5,74286.0,33963.0,170.0,3.258578
1,08520,27468.0,37.4,90293.0,37175.0,1834.0,6.676860
2,08525,4782.0,47.1,118656.0,59848.0,43.0,0.899205
3,08527,54867.0,42.2,88588.0,37021.0,2191.0,3.993293
4,08528,245.0,48.5,58676.0,49117.0,0.0,0.000000


In [65]:
census_pd.head()

NameError: name 'census_pd' is not defined

In [10]:
male_data=c1.acs1.get('B01003_001E', {'for': 'state:*'})
m_pd = pd.DataFrame(male_data)
m_pd = m_pd.rename(columns={"B01003_001E": "Population", 
                                       "state": "State Code"})
m_pd = m_pd[["State Code","Population"]]
m_pd=m_pd.fillna(0)
m_pd.head()
m_pd.to_csv("US_Population.csv", encoding="utf-8", index=False)

In [8]:
male_data=c1.acs1.get('B01003_001E', {'for': 'state:*'})
m_pd = pd.DataFrame(male_data)
m_pd = m_pd.rename(columns={"B24010_019E": "Population", 
                                       "state": "State Code"})
m_pd = m_pd[["State Code","Population"]]
m_pd.head()

KeyError: "['Population'] not in index"

In [7]:
female_data=c1.acs1.get('B24010_155E', {'for': 'state:*'})
f_pd = pd.DataFrame(female_data)
f_pd = f_pd.rename(columns={"B24010_155E": "Female", 
                                       "state": "State Code"})
f_pd = f_pd[["State Code","Female"]]
f_pd.head()

,State Code,Female
0,01,123251.0
1,02,21991.0
2,04,221393.0
3,05,72013.0
4,06,1348070.0


,State Code,Female
0,01,66276.0
1,02,NaN
2,04,52836.0
3,05,44928.0
4,06,421201.0


In [8]:
# get state names
state_dict = us.states.mapping('fips', 'name')
# Append DataFrame
mf_df = pd.merge(f_pd, m_pd, on='State Code', how='inner')
mf_df=mf_df.fillna(0)

mf_df["Total"] = mf_df["Male"].astype(int) + mf_df["Female"].astype(int)
mf_df["State"] = ''
mf_df["Type"] = ''
#mf_df["State"] = state_dict[mf_df["State Code"]] 
state_dict = us.states.mapping('fips', 'name')
for index, row in mf_df.iterrows():
    mf_df.loc[index,'State'] = state_dict[mf_df.loc[index,'State Code']]
    mf_df.loc[index,'Type'] = 'Manufacturing'


mf_df = mf_df[["State Code", "State", "Type","Male", "Female","Total"]]
mf_df.sort_values('Total', ascending=False).head()

NameError: name 'm_pd' is not defined

In [2]:
def func_merge_df(df1, df2, column):
    mf_df = pd.merge(df1, df2, on=column, how='inner')
    return mf_df

def func_getdata(key1, key2, jobtype):
    male_data= []
    female_data = []
    state_dict = us.states.mapping('fips', 'name')
    
    male_data=c1.acs1.get(key1, {'for': 'state:*'})
    female_data=c1.acs1.get(key2, {'for': 'state:*'})
    m_pd = pd.DataFrame(male_data)
    f_pd = pd.DataFrame(female_data)
    # Merge the dataframe
    mf_df = func_merge_df(df1=f_pd, df2=m_pd, column='state')
    mf_df=mf_df.fillna(0)
    # Get the total 
    mf_df["State Name"] = ''
    mf_df["Type"] = ''
    # Get the State name
    for index, row in mf_df.iterrows():
        mf_df.loc[index,'State Name'] = state_dict[mf_df.loc[index,'state']]
        mf_df.loc[index,'Type'] = jobtype
    return mf_df

   
#rs_data = func_getdata(key1 = 'B24040_007E', key2 = 'B24040_111E', jobtype = 'Manufacturing')
#rs_data = rs_data.rename(columns={"B24040_007E": "Male","B24040_111E":"Female","state": "State Code"})
#rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
#rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
#rs_data.sort_values('Total', ascending=False).head() 
#rs_data

In [14]:
cnt = 0
input_data=[{'Male':'B24040_007E','Female':'B24040_111E','jobtype':'Manufacturing'},
           {'Male':'B24010_019E','Female':'B24010_155E','jobtype':'IT'},
           {'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}
          ]
input_df = pd.DataFrame(input_data)

for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    rs_data.sort_values('Total', ascending=False).head() 
    if cnt !=0:
        cnt = cnt+1

# First time for the header
    if cnt == 0:
        rs_data.to_csv("job_census_data.csv", encoding="utf-8", index=False)
        cnt = cnt + 1 

    if cnt > 1:
        with open('job_census_data.csv', 'a') as f:
            rs_data.to_csv(f, header=False, index=False)

print("Job Completed")

Job Completed


In [3]:
#top states.
cnt = 0

input_data=[{'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'}]
#input_data=[{'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}]
input_df = pd.DataFrame(input_data)

for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    final_df = rs_data.sort_values('Total', ascending=False).head() 
    
final_df

,State Code,State Name,Type,Male,Female,Total
4,06,California,Social Workers,66326.0,185081.0,251407
32,36,New York,Social Workers,45515.0,137849.0,183364
43,48,Texas,Social Workers,28644.0,105919.0,134563
38,42,Pennsylvania,Social Workers,29250.0,74910.0,104160
9,12,Florida,Social Workers,23239.0,76424.0,99663


In [4]:
# Social Workers
cnt = 1
input_data=[{'Male':'B24010_066E','Female':'B24010_217E','jobtype':'Psychiatric'},
           {'Male':'B24010_038E','Female':'B24010_189E','jobtype':'Religious'},
           {'Male':'B24010_037EA','Female':'B24010_188E','jobtype':'Counsolers'},
           {'Male':'B24010_091E','Female':'B24010_242E','jobtype':'Child Care'}
          ]
input_df = pd.DataFrame(input_data)
input_df
for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    rs_data.sort_values('Total', ascending=False).head() 

# First time for the header
    if cnt == 1:
        rs_data.to_csv("job_SocialWorker_data.csv", encoding="utf-8", index=False)
        cnt = cnt + 1

    if cnt > 1:
        with open('job_SocialWorker_data.csv', 'a') as f:
            rs_data.to_csv(f, header=False, index=False)

print("Job Completed")

Job Completed


In [5]:
# Social Workers
cnt = 1
input_data=[{'Male':'B24010_066E','Female':'B24010_217E','jobtype':'Psychiatric'},
           {'Male':'B24010_038E','Female':'B24010_189E','jobtype':'Religious'},
           {'Male':'B24010_037EA','Female':'B24010_188E','jobtype':'Counsolers'},
           {'Male':'B24010_091E','Female':'B24010_242E','jobtype':'Child Care'}
          ]
input_df = pd.DataFrame(input_data)
input_df
for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    
rs_data.sort_values('Total', ascending=False).head() 

,State Code,State Name,Type,Male,Female,Total
4,06,California,Child Care,14948.0,153040.0,167988
32,36,New York,Child Care,6629.0,107009.0,113638
43,48,Texas,Child Care,7149.0,98652.0,105801
13,17,Illinois,Child Care,2778.0,54157.0,56935
9,12,Florida,Child Care,3887.0,49141.0,53028


In [9]:
female_data=c1.acs1.get('B24010_189E', {'for': 'state:*'})
f_pd = pd.DataFrame(female_data)
f_pd = f_pd.rename(columns={"B24010_189E": "Female", 
                                       "state": "State Code"})
f_pd = f_pd[["State Code","Female"]]
f_pd.head()

,State Code,Female
0,01,3609.0
1,02,551.0
2,04,3209.0
3,05,1972.0
4,06,20834.0


In [11]:
# Finance
cnt = 1
input_data=[{'Male':'B24010_105E','Female':'B24010_256E','jobtype':'Audit Clerk'},
           {'Male':'B24010_016E','Female':'B24010_167E','jobtype':'Specialist'},
           {'Male':'B24010_008E','Female':'B24010_159E','jobtype':'Finance Manager'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Accountants'}
          ]
input_df = pd.DataFrame(input_data)
input_df
for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    rs_data.sort_values('Total', ascending=False).head() 

# First time for the header
    if cnt == 1:
        rs_data.to_csv("job_Finance_data.csv", encoding="utf-8", index=False)
        cnt = cnt + 1

    if cnt > 1:
        with open('job_Finance_data.csv', 'a') as f:
            rs_data.to_csv(f, header=False, index=False)

print("Job Completed")

Job Completed


In [4]:
# Finance
cnt = 1
input_data=[{'Male':'B24010_105E','Female':'B24010_256E','jobtype':'Audit Clerk'},
           {'Male':'B24010_016E','Female':'B24010_167E','jobtype':'Specialist'},
           {'Male':'B24010_008E','Female':'B24010_159E','jobtype':'Finance Manager'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Accountants'}
          ]
 
key1_val = 'B24010_105E'
key2_val = 'B24010_256E'
jobtypeVal = 'Audit Clerk'

rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
rs_data
rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
#rs_data.sort_values('Total', ascending=False).head() 
filter_df = rs_data[rs_data['State Name']=='California']
filter_df

,State Code,State Name,Type,Male,Female,Total
4,06,California,Audit Clerk,26429.0,109060.0,135489


In [6]:
#top states
cnt = 0
input_data=[{'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'}]
#input_data=[{'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}]
input_df = pd.DataFrame(input_data)
for index, row in input_df.iterrows():
   filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
   key1_val = ','.join(str(v) for v in filter_df['Male'])
   key2_val = ','.join(str(v) for v in filter_df['Female'])
   jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
   rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
   rs_data
   rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
   rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
   rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
   final_df = rs_data.sort_values('Total', ascending=False).head()
final_df

,State Code,State Name,Type,Male,Female,Total
4,06,California,Social Workers,66326.0,185081.0,251407
32,36,New York,Social Workers,45515.0,137849.0,183364
43,48,Texas,Social Workers,28644.0,105919.0,134563
38,42,Pennsylvania,Social Workers,29250.0,74910.0,104160
9,12,Florida,Social Workers,23239.0,76424.0,99663
